# Final Validation

After previous sampling, I realised that training data should not be used to test the model.

In [164]:
#WARNING: This scrapes live data. Only run this when you are absolutely sure you want to scrape current data!
#I previously scraped this so I will go straight to loading the file in the next cell

request_params = dict()
request_params['filter'] = 'recent'
request_params['language'] = 'english'
request_params['day_range'] = '365'

app_id = 779340
review_dict, query_count = steamreviews.download_reviews_for_app_id(app_id, chosen_request_params=request_params)

[appID = 779340] expected #reviews = 8462


In [189]:
#read in a json file
with open('./data/review_779340_2.json') as jsonfile:
    jayson = json.load(jsonfile)

In [190]:
review_data = [v for v in jayson['reviews'].values()]
author_key_list = list(review_data[0]['author'].keys())

In [191]:
len(review_data)

8422

In [192]:
datetime.datetime.utcfromtimestamp(min([review_data[i]['timestamp_created'] for i in range(len(review_data))])).strftime('%Y-%m-%d')

'2019-05-23'

In [193]:
datetime.datetime.utcfromtimestamp(max([review_data[i]['timestamp_created'] for i in range(len(review_data))])).strftime('%Y-%m-%d')

'2019-07-20'

In [194]:
#previous max value was 1562591773; 2019-07-08
max([review_data[i]['timestamp_created'] for i in range(len(review_data))])

1563618842

In [195]:
#for definition of each field in the JSON files, visit (here)[https://partner.steamgames.com/doc/store/getreviews]

steam_dict = {'recommendationid':[review_data[i]['recommendationid'] for i in range(len(review_data))],
              
              'author_steamid':[review_data[i]['author']['steamid'] for i in range(len(review_data))],
              
              'author_num_games_owned':[review_data[i]['author']['num_games_owned'] for i in range(len(review_data))],
              
              'author_num_reviews':[review_data[i]['author']['num_reviews'] for i in range(len(review_data))],
              
              'author_playtime_forever':[review_data[i]['author']['playtime_forever'] for i in range(len(review_data))], 
              #data recorded in minutes - independently verified: https://steamcommunity.com/id/mydogsnameissnowballball/recommended/730/ - 43586 divided by 724.6h on record = 60 min
              
              'author_playtime_last_two_weeks':[review_data[i]['author']['playtime_last_two_weeks'] for i in range(len(review_data))],
              
              'author_last_played':[review_data[i]['author']['last_played'] for i in range(len(review_data))],
              
              'review':[review_data[i]['review'] for i in range(len(review_data))],
              
              'timestamp_created':[review_data[i]['timestamp_created'] for i in range(len(review_data))],
              
              'timestamp_updated':[review_data[i]['timestamp_updated'] for i in range(len(review_data))],
              
              'voted_up':[review_data[i]['voted_up'] for i in range(len(review_data))],
              
              'weighted_vote_score':[review_data[i]['weighted_vote_score'] for i in range(len(review_data))],
              
              'steam_purchase':[review_data[i]['steam_purchase'] for i in range(len(review_data))],
              
              'received_for_free':[review_data[i]['received_for_free'] for i in range(len(review_data))],
              
              'written_during_early_access':[review_data[i]['written_during_early_access'] for i in range(len(review_data))]}

In [196]:
df_raw = pd.DataFrame(steam_dict)
df_raw.head(2)

,recommendationid,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_last_played,review,timestamp_created,timestamp_updated,voted_up,weighted_vote_score,steam_purchase,received_for_free,written_during_early_access
0,54192944,76561198859964653,57,24,1694,1694,1563886009,"Its a chess within a chess, two layers of comp...",1563618842,1563810779,True,0.495412856340408325,True,False,False
1,54192696,76561198433597055,2,1,7441,4943,1563870098,Please give us more Chinese theme game.I will ...,1563617491,1563617491,True,0.523809552192687988,True,False,False


In [197]:
len(df_raw['recommendationid'].unique())

8422

In [198]:
df_raw = df_raw[df_raw['received_for_free']==False] # Redundant for F2P games, because nobody can buy them

In [201]:
df_raw.dtypes

recommendationid                  object
author_steamid                    object
author_num_games_owned             int64
author_num_reviews                 int64
author_playtime_forever            int64
author_playtime_last_two_weeks     int64
author_last_played                 int64
review                            object
timestamp_created                  int64
timestamp_updated                  int64
voted_up                            bool
weighted_vote_score               object
steam_purchase                      bool
received_for_free                   bool
written_during_early_access         bool
dtype: object

In [203]:
df_raw = df_raw[df_raw['timestamp_created']>1562591773]

In [204]:
len(df_raw)

206

In [205]:
df_raw.describe()

,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_last_played,timestamp_created,timestamp_updated
count,206.000000,206.000000,206.000000,206.000000,2.060000e+02,2.060000e+02,2.060000e+02
mean,136.820388,12.461165,4731.038835,1242.344660,1.563050e+09,1.563089e+09,1.563102e+09
std,157.885700,20.120514,4680.173901,1543.339617,1.106116e+06,2.940085e+05,3.000377e+05
min,1.000000,1.000000,30.000000,0.000000,1.558703e+09,1.562595e+09,1.562595e+09
25%,36.250000,2.000000,1318.000000,33.500000,1.562953e+09,1.562823e+09,1.562849e+09
50%,96.000000,5.000000,3315.500000,688.000000,1.563451e+09,1.563075e+09,1.563089e+09
75%,179.750000,14.000000,6401.000000,1872.000000,1.563786e+09,1.563361e+09,1.563374e+09
max,1417.000000,142.000000,24184.000000,9085.000000,1.563886e+09,1.563619e+09,1.563811e+09


In [206]:
df_raw['author_playtime_forever']=df_raw['author_playtime_forever']/60
df_raw['author_playtime_last_two_weeks']=df_raw['author_playtime_last_two_weeks']/60

In [207]:
df_raw_num = df_raw.select_dtypes(exclude=['O','bool'])

In [208]:
df_raw_num_high_play = df_raw_num[(df_raw_num['author_playtime_forever']>2) & (df_raw_num['author_playtime_last_two_weeks']>=0.5)]

In [209]:
len(df_raw_num_high_play)

150

In [210]:
df_usable = df_raw[(df_raw['author_playtime_forever']>=3)& (df_raw_num['author_playtime_last_two_weeks']>=(10/60))] #recent opinions are more important, and user must have played longer than refundable playtime cutoff
df_usable.head(2)

,recommendationid,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_last_played,review,timestamp_created,timestamp_updated,voted_up,weighted_vote_score,steam_purchase,received_for_free,written_during_early_access
0,54192944,76561198859964653,57,24,28.233333,28.233333,1563886009,"Its a chess within a chess, two layers of comp...",1563618842,1563810779,True,0.495412856340408325,True,False,False
1,54192696,76561198433597055,2,1,124.016667,82.383333,1563870098,Please give us more Chinese theme game.I will ...,1563617491,1563617491,True,0.523809552192687988,True,False,False


In [211]:
review_lengths = [len(x.split()) for x in df_usable['review']]
print(len(review_lengths)) #no of reviews
print(sum(review_lengths)/len(review_lengths)) #mean character length of a review
df_usable['review_length']=df_usable['review'].map(lambda x: len(x.split()))

153
82.359477124183


C:\Users\Alfred\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [212]:
df_ready = df_usable[df_usable['review_length']>np.percentile(df_usable['review_length'],25)].reset_index(drop=True) #Remove very short reviews that are likely to be junk

In [214]:
print(min(df_ready['review_length'])) #shortest review length, in words. space-delimited
df_ready.head(2)

12


,recommendationid,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_last_played,review,timestamp_created,timestamp_updated,voted_up,weighted_vote_score,steam_purchase,received_for_free,written_during_early_access,review_length
0,54183390,76561197984368747,216,6,69.933333,0.300000,1563395139,Awesome. Great game! \n\nDo wish they gave you...,1563562610,1563562610,True,0,False,False,False,80
1,54183023,76561198451245786,155,4,58.450000,2.566667,1563617150,became emperor of china then everyone betrayed...,1563560850,1563560850,True,0,False,False,False,14


In [216]:
#drop duplicate entries in reviews, because they are considered spam
df_cleaning = df_ready.drop_duplicates(subset='review')

In [217]:
df_cleaning.select_dtypes(include='O').describe()

,recommendationid,author_steamid,review,weighted_vote_score
count,111,111,111,111
unique,111,111,111,49
top,54032935,76561198007691650,I'm usually not a fan of total war but this ga...,0
freq,1,1,1,53


In [218]:
deduplicated_rows = abs(len(df_cleaning) - len(df_ready)) #find out how many rows were removed during deduplication
print(deduplicated_rows)
print(deduplicated_rows/len(df_ready)) #Percentage of data that was duplicated

0
0.0


In [221]:
#get languages by document
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)
    
def get_lang_doc(text):
    doc = nlp(text)
    return doc._.language['language']

#count languages for sentences in document -- decided not to use this because 1. it slows computation a lot 2. it throws a lot of false positives, especially when splitting short lines
def get_langs_sent(text):
    langs={}
    lang_st={}
    doc = nlp(text)
    
    for i, sent in enumerate(doc.sents):
        langs[i]=sent._.language['language'] #Iterating through sentence level, thus only need to add language. Probability scores should be quite high per sentence.
    
    lang_st = list(set(langs.values())) #Get language code of each language that appeared. As long as more than 1 language appears, the review will be excluded.
    
    return lang_st

In [222]:
#No repeated reviews.

doc_langs = [get_lang_doc(x) for x in df_cleaning['review']] #WARNING: SLOW! May take a couple of minutes to finish.
df_cleaning['review_lang']=doc_langs

In [223]:
df_cleaning.to_csv('./dataframes/vldtn_test_cleaning.csv') #saving so I don't have to run the cleaning operation again

In [224]:
df_cleaning = pd.read_csv('./dataframes/vldtn_test_cleaning.csv',index_col=0)

In [225]:
df_langs_cleaned = df_cleaning[df_cleaning['review_lang']=='en'].reset_index(drop=True)
len(df_langs_cleaned)

111

In [226]:
mult_ln_reviews_dropped = (len(df_cleaning) - len(df_langs_cleaned))/len(df_cleaning)
print(mult_ln_reviews_dropped)

0.0


In [229]:
parser = bbcode.Parser()

def parse(review):
    parsed_text = parser.strip(review)
    return parsed_text

#open pre-defined dictionary of english contractions, saved separately to avoid clutter (long dictionary, can be loaded as dict using json package)
with open('./en_contractions/contra_dict.txt') as contra_dict:
    cList = json.load(contra_dict)

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text.lower())

In [230]:
def parse_sent_tokenize(review):
    
    sent_tokens = [] #store sentence tokens here
    
    text = parse(review)  #remove BBcode first
    
    text = expandContractions(text)
    
    #All word lengths should be >1 character and <= length of the longest word in the English language. It's common for people spam incoherent letters on the Internet. Do not remove stopwords yet.
    text = [word for word in text.split() if len(word)>1 and len(word) <= len('pneumonoultramicroscopicsilicovolcanoconiosis')] 
   
    parsed_text = ' '.join(word for word in text)
    
    doc = nlp(parsed_text)
    for i,sent in enumerate(doc.sents):
        sent_tokens.append(sent.text)
    
    return sent_tokens

#remove punctuation
def remove_punc(sents):
    clean_text = [s.translate(str.maketrans('', '', string.punctuation)) 
                  for s in sents]
    return clean_text

In [232]:
df_langs_cleaned.head(2)

,recommendationid,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_last_played,review,timestamp_created,timestamp_updated,voted_up,weighted_vote_score,steam_purchase,received_for_free,written_during_early_access,review_length,review_lang
0,54183390,76561197984368747,216,6,69.933333,0.300000,1563395139,Awesome. Great game! \n\nDo wish they gave you...,1563562610,1563562610,True,0.0,False,False,False,80,en
1,54183023,76561198451245786,155,4,58.450000,2.566667,1563617150,became emperor of china then everyone betrayed...,1563560850,1563560850,True,0.0,False,False,False,14,en


In [554]:
big_sent_lst =[] #For storing ALL sentences in the dataset. Doesn't matter which reviews they came from

review_index_lst = [] #for storing all index numbers for each sentence so we can identify which reviews they belong to and thus group scores by topics and reviews

sent_df = pd.DataFrame() #For visualisation of sentence transformations and saving of data

review_count = 0
for review in df_langs_cleaned['review']:
    sent_tokens = parse_sent_tokenize(review)
    sent_tokens = remove_punc(sent_tokens)
    big_sent_lst.extend(sent_tokens)
    
    review_count+=1
    review_num_lst = [str(review_count)] * len(sent_tokens) #for each review, create list of corresponding review index numbers that is as long as the number of sentences for easier groupby
    review_index_lst.extend(review_num_lst)

sent_df['original_sent'] = [x for x in big_sent_lst]
sent_df['test_set_review_number'] = review_index_lst
sent_df['token_sentences'] = [x.split() for x in big_sent_lst]

sent_df.head(2)

,original_sent,test_set_review_number,token_sentences
0,awesome,1,[awesome]
1,great game do wish they gave you more factions...,1,"[great, game, do, wish, they, gave, you, more,..."


In [555]:
#load dataset for training ngram phraser

final_data = pd.read_csv('./dataframes/dominant_topics.csv',index_col=0)

In [556]:
final_data.head()

,dominant_topic,original_text,topic_perc_contrib,tokens,num_tokens,unique_tokens,num_unique_tokens,keywords
0,Tactical Gameplay,Well for me game still tons of work. i like it...,0.4715,"['ton', 'diplomacy', 'joke', 'military_access'...",85,"['guess', 'artillery', 'chain', 'siege', 'sent...",57,"unit, battle, faction, army, general, characte..."
1,Characters,I pursued Lu Bu. Now I [b]AM[/b] LU BU.,0.7433,"['lu_bu', 'lu_bu']",2,['lu_bu'],1,"china, war, cao_cao, lu_bu, liu_bei, yuan_shao..."
2,Series Authenticity,Absolutely great game. \nAll the new diplomacy...,0.9391,"['diplomacy_options', 'depth', 'unit_variety',...",10,"['warhammer', 'depth', 'combat', 'rt', 'diplom...",9,"three_kingdoms, series, fan, diplomacy, battle..."
3,Series Authenticity,A fine blend of Warhammer I/II: Total War and ...,0.6487,"['blend', 'warhammer_three', 'shogun_two', 'lo...",20,"['blend', 'warhammer_three', 'character', 'bat...",20,"three_kingdoms, series, fan, diplomacy, battle..."
4,Series Authenticity,Innovative Total Game that has lots of persona...,0.7290,"['personality', 'diplomacy', 'idea', 'combat',...",11,"['art_style', 'paid_dlc', 'combat', 'content',...",10,"three_kingdoms, series, fan, diplomacy, battle..."


In [557]:
#initialise BBcode parser and spacy NLP for parsing and tokenising sentences

nlp = spacy.load("en_core_web_sm")
parser = bbcode.Parser()

def parse(review):
    parsed_text = parser.strip(review)
    return parsed_text

#open pre-defined dictionary of english contractions, saved separately to avoid clutter (long dictionary, can be loaded as dict using json package)
with open('./en_contractions/contra_dict.txt') as contra_dict:
    cList = json.load(contra_dict)

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text.lower())

In [558]:
big_sent_lst2 =[] #For storing ALL sentences in the dataset. Doesn't matter which reviews they came from
train_sent_df = pd.DataFrame() #For visualisation of sentence transformations and saving of data

for review in final_data['original_text']:
    sent_tokens = parse_sent_tokenize(review)
    sent_tokens = remove_punc(sent_tokens)
    big_sent_lst2.extend(sent_tokens)

In [559]:
train_sent_df['original_text'] = [x for x in big_sent_lst2]
train_sent_df['token_sentences'] = [x.split() for x in big_sent_lst2]
train_sent_df.head()

,original_text,token_sentences
0,well for me game still tons of work,"[well, for, me, game, still, tons, of, work]"
1,like it but just cant recommend,"[like, it, but, just, cant, recommend]"
2,1,[1]
3,diplomacy,[diplomacy]
4,it is joke,"[it, is, joke]"


In [560]:
def num2word(d):
    
    if (len(d) == 1 and d in '0123')  or (d in ['ii','iii']):
        word = num_dict[d]
    
    elif (len(str(d))==1 and str(d) in '0123'):
        word = num_dict(str(d))
    
    else:
        word = d
    
    return word

en_stopwords = list(set(STOPWORDS))
en_stopwords.extend(['good','better','great','lot','game','like','I','i'])
en_stopwords = [w for w in en_stopwords if w not in ['one','two','three']]

def remove_stopwords(doc):
    words = [num2word(w) for w in doc if w != '' and w not in en_stopwords]
    return words

train_sent_df['sents_unstopped'] = train_sent_df['token_sentences'].map(lambda x: remove_stopwords(x))

bigram = gensim.models.Phrases(list(train_sent_df['sents_unstopped']), min_count=5, threshold=10) # feed a list of lists of words e.g. [['word1','word2'],['word3','word4'] to get bigrams]
trigram = gensim.models.Phrases(bigram[list(train_sent_df['sents_unstopped'])], threshold=10)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

#at this point we will make n-grams, then lemmatise using spacy since it can go by permitted postags

def spacy_lemma(bow,allowed_postags=['NOUN']):
    
    lemma_doc = nlp(" ".join(bow)) 

    lemma_text = [token.text if '_' in token.text else token.lemma_ if token.pos_ in allowed_postags else '' for token in lemma_doc]
    
    return lemma_text

In [561]:
#WARNING! this cell can be a little slow, especially with larger datasets (it already takes ~3 minutes with 3600+ rows) -- will consider stemming as a future modification

sent_df['sents_unstopped'] = sent_df['token_sentences'].map(lambda x: remove_stopwords(x))
sent_df['3gram_sentences'] = make_trigrams(sent_df['sents_unstopped'])

en_stopwords.extend(['good','better','great','lot','game','like','I','i','one','two','three','thing','bit','total_war','time'])

sent_df['3gram_sentences_lemma'] = sent_df['3gram_sentences'].map(lambda x: spacy_lemma(x))
sent_df['3gram_sentences_clean'] = sent_df['3gram_sentences_lemma'].map(lambda x: remove_stopwords(x))
sent_df.head(10)

,original_sent,test_set_review_number,token_sentences,sents_unstopped,3gram_sentences,3gram_sentences_lemma,3gram_sentences_clean
0,awesome,1,[awesome],[awesome],[awesome],[],[]
1,great game do wish they gave you more factions...,1,"[great, game, do, wish, they, gave, you, more,...","[wish, gave, factions, starting, map, options]","[wish, gave, factions, starting, map, options]","[wish, , faction, , map, option]","[wish, faction, map, option]"
2,the whole bottom of the map is empty which mea...,1,"[the, whole, bottom, of, the, map, is, empty, ...","[map, means, wu, takes, free, territory]","[map, means, wu, takes, free, territory]","[map, , , , , territory]","[map, territory]"
3,really enjoy how romance of the three kingdoms...,1,"[really, enjoy, how, romance, of, the, three, ...","[enjoy, romance, three, kingdoms, lets, play, ...","[enjoy, romance_three_kingdoms, lets, play, wa...","[, romance_three_kingdoms, let, , , , , starti...","[romance_three_kingdoms, let, starting, map, o..."
4,those are minor complaints to great and amazin...,1,"[those, are, minor, complaints, to, great, and...","[minor, complaints, amazing]","[minor, complaints, amazing]","[, complaint, ]",[complaint]
5,my favorite total war series to date and have ...,1,"[my, favorite, total, war, series, to, date, a...","[favorite, total, war, series, date, pretty, p...","[favorite_total_war, series, date, pretty, pla...","[favorite_total_war, series, date, , , medievial]","[favorite_total_war, series, date, medievial]"
6,became emperor of china then everyone betrayed...,2,"[became, emperor, of, china, then, everyone, b...","[emperor, china, betrayed, 1010, reunite, china]","[emperor, china, betrayed, 1010, reunite, china]","[emperor, china, , , reunite, china]","[emperor, china, reunite, china]"
7,graphics you forget what reality is beautiful ...,3,"[graphics, you, forget, what, reality, is, bea...","[graphics, forget, reality, beautiful, decent,...","[graphics, forget, reality, beautiful, decent,...","[graphic, , reality, , , , don‘t, , , paintexe...","[graphic, reality, don‘t, paintexe, gameplay, ..."
8,audience kids teens adults average solitare pl...,3,"[audience, kids, teens, adults, average, solit...","[audience, kids, teens, adults, average, solit...","[audience, kids, teens, adults, average, solit...","[audience, kid, teen, adult, , , player, pc, r...","[audience, kid, teen, adult, player, pc, requi..."
9,decent fast rich boiiiiii ask nasa if they hav...,3,"[decent, fast, rich, boiiiiii, ask, nasa, if, ...","[decent, fast, rich, boiiiiii, ask, nasa, spar...","[decent, fast, rich, boiiiiii, ask, nasa, spar...","[, , , boiiiiii, , nasa, , difficulity, press,...","[boiiiiii, nasa, difficulity, press, ‚, brain,..."


In [562]:
sent_df['num_tokens']=sent_df['3gram_sentences_clean'].map(lambda x: len(x))
sent_df.head()

,original_sent,test_set_review_number,token_sentences,sents_unstopped,3gram_sentences,3gram_sentences_lemma,3gram_sentences_clean,num_tokens
0,awesome,1,[awesome],[awesome],[awesome],[],[],0
1,great game do wish they gave you more factions...,1,"[great, game, do, wish, they, gave, you, more,...","[wish, gave, factions, starting, map, options]","[wish, gave, factions, starting, map, options]","[wish, , faction, , map, option]","[wish, faction, map, option]",4
2,the whole bottom of the map is empty which mea...,1,"[the, whole, bottom, of, the, map, is, empty, ...","[map, means, wu, takes, free, territory]","[map, means, wu, takes, free, territory]","[map, , , , , territory]","[map, territory]",2
3,really enjoy how romance of the three kingdoms...,1,"[really, enjoy, how, romance, of, the, three, ...","[enjoy, romance, three, kingdoms, lets, play, ...","[enjoy, romance_three_kingdoms, lets, play, wa...","[, romance_three_kingdoms, let, , , , , starti...","[romance_three_kingdoms, let, starting, map, o...",5
4,those are minor complaints to great and amazin...,1,"[those, are, minor, complaints, to, great, and...","[minor, complaints, amazing]","[minor, complaints, amazing]","[, complaint, ]",[complaint],1


In [563]:
test_sent_df = sent_df[sent_df['num_tokens']>1].reset_index(drop=True)
dropped_rows = abs(len(sent_df)-len(test_sent_df))/len(sent_df)
print(dropped_rows)
print(len(test_sent_df))

0.328
504


In [564]:
#Proportion of test set sentences vs the training set

497/10961

0.04534257823191315

In [565]:
#In hindsight, I should have reserved 20% of the data for testing.

test_sent_df.describe()

,num_tokens
count,504.000000
mean,4.146825
std,2.777048
min,2.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,26.000000


In [566]:
test_sent_df.to_csv('./dataframes/vldtn_test_df.csv')

In [567]:
test_sent_df = pd.read_csv('./dataframes/vldtn_test_df.csv',index_col=0)

In [568]:
test_sent_df.head(2)

,original_sent,test_set_review_number,token_sentences,sents_unstopped,3gram_sentences,3gram_sentences_lemma,3gram_sentences_clean,num_tokens
0,great game do wish they gave you more factions...,1,"['great', 'game', 'do', 'wish', 'they', 'gave'...","['wish', 'gave', 'factions', 'starting', 'map'...","['wish', 'gave', 'factions', 'starting', 'map'...","['wish', '', 'faction', '', 'map', 'option']","['wish', 'faction', 'map', 'option']",4
1,the whole bottom of the map is empty which mea...,1,"['the', 'whole', 'bottom', 'of', 'the', 'map',...","['map', 'means', 'wu', 'takes', 'free', 'terri...","['map', 'means', 'wu', 'takes', 'free', 'terri...","['map', '', '', '', '', 'territory']","['map', 'territory']",2


In [569]:
test_sent_df.dtypes

original_sent             object
test_set_review_number     int64
token_sentences           object
sents_unstopped           object
3gram_sentences           object
3gram_sentences_lemma     object
3gram_sentences_clean     object
num_tokens                 int64
dtype: object

In [570]:
test_sent_df['token_sentences'] = test_sent_df['token_sentences'].map(lambda x: ''.join(c for c in x if c=='_' or c not in string.punctuation).split()) #n-grams underscores must be preserved for readability
test_sent_df['sents_unstopped'] = test_sent_df['sents_unstopped'].map(lambda x: ''.join(c for c in x if c=='_' or c not in string.punctuation).split()) 
test_sent_df['3gram_sentences'] = test_sent_df['3gram_sentences'].map(lambda x: ''.join(c for c in x if c=='_' or c not in string.punctuation).split()) 
test_sent_df['3gram_sentences_clean'] = test_sent_df['3gram_sentences_clean'].map(lambda x: ''.join(c for c in x if c=='_' or c not in string.punctuation).split()) 

In [571]:
test_sent_df.head(2)

,original_sent,test_set_review_number,token_sentences,sents_unstopped,3gram_sentences,3gram_sentences_lemma,3gram_sentences_clean,num_tokens
0,great game do wish they gave you more factions...,1,"[great, game, do, wish, they, gave, you, more,...","[wish, gave, factions, starting, map, options]","[wish, gave, factions, starting, map, options]","['wish', '', 'faction', '', 'map', 'option']","[wish, faction, map, option]",4
1,the whole bottom of the map is empty which mea...,1,"[the, whole, bottom, of, the, map, is, empty, ...","[map, means, wu, takes, free, territory]","[map, means, wu, takes, free, territory]","['map', '', '', '', '', 'territory']","[map, territory]",2


In [572]:
documents = list(test_sent_df['3gram_sentences_clean'])
dictionary = gensim.corpora.Dictionary(documents)

dictionary.filter_extremes(no_below = 5,no_above=0.5)     

corpus = [dictionary.doc2bow(text) for text in documents]

In [573]:
topic_list = lda_model.print_topics(num_topics=5, num_words=15)
for index, i in enumerate(topic_list):
    str1 = str(i[1])
    for c in "0123456789+*\".":
        str1 = str1.replace(c, "")
    str1 = str1.replace("  ", " ")
    print(str1)

three_kingdoms series fan diplomacy battle love best_total_war campaign total_war_games fun character shogun play title gameplay
china war cao_cao lu_bu liu_bei yuan_shao army character man emperor warlord friend history faction sun_jian
faction army diplomacy vassal war player people way city problem campaign option spy three_kingdoms review
unit battle faction army general character diplomacy campaign hero building duel enemy way map combat
campaign hour issue battle bug crash multiplayer review rome fix play patch work month release


In [574]:
#credit to https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/ for this cell's code, which I modified to suit my needs

topic_dict = {'0':'Content & Authenticity',
              '1':'Characters',
              '2':'Strategic Gameplay',
              '3':'Tactical Gameplay',
              '4':'UI/UX, Performance, Stability',
             }

#created a dictionary so I can show topic names instead of numbers, without any complicated code

def format_topics_sentences(ldamodel=lda_model, corpus=corpus, documents=documents):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([topic_dict[str(topic_num)], round(prop_topic,4), topic_keywords]), ignore_index=True)   #replaced int(topic_num) with
                
            else:
                break
                
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    orig_contents = pd.Series(test_sent_df['original_sent'])
    docs = pd.Series(documents)
    sent_topics_df = pd.concat([sent_topics_df,docs, orig_contents], axis=1)
    return(sent_topics_df)

In [575]:
#generate dataframe
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, documents=documents)

In [576]:
# Format dataframe
df_sent_dom_topic = df_topic_sents_keywords.reset_index()

df_sent_dom_topic.columns = ['document_no', 'dominant_topic', 'topic_perc_contrib', 'keywords', 'tokens','original_text']

df_sent_dom_topic['num_tokens']=df_sent_dom_topic['tokens'].map(lambda x: len(x))
df_sent_dom_topic['review_number'] = test_sent_df['test_set_review_number']

df_sent_dom_topic = df_sent_dom_topic[['review_number','dominant_topic','original_text','topic_perc_contrib','tokens','num_tokens','keywords']]

# Show dataframe
df_sent_dom_topic.head(5)

,review_number,dominant_topic,original_text,topic_perc_contrib,tokens,num_tokens,keywords
0,1,Strategic Gameplay,great game do wish they gave you more factions...,0.7929,"[wish, faction, map, option]",4,"faction, army, diplomacy, vassal, war, player,..."
1,1,Strategic Gameplay,the whole bottom of the map is empty which mea...,0.5874,"[map, territory]",2,"faction, army, diplomacy, vassal, war, player,..."
2,1,Strategic Gameplay,really enjoy how romance of the three kingdoms...,0.7243,"[romance_three_kingdoms, let, starting, map, o...",5,"faction, army, diplomacy, vassal, war, player,..."
3,1,Tactical Gameplay,my favorite total war series to date and have ...,0.5686,"[favorite_total_war, series, date, medievial]",4,"unit, battle, faction, army, general, characte..."
4,2,Tactical Gameplay,became emperor of china then everyone betrayed...,0.7038,"[emperor, china, reunite, china]",4,"unit, battle, faction, army, general, characte..."


In [577]:
df_sent_dom_topic.to_csv('./dataframes/vldtn_test_sent_topics.csv')

In [578]:
make_trigrams([['shogun','two','shogun','2']])

[['shogun_two', 'shogun', '2']]

In [579]:
spacy_lemma(['map'])

['']

In [581]:
spacy_lemma(['crashing'])

['']

In [582]:
spacy_lemma(['crash'])

['crash']

In [580]:
df_sent_dom_topic[30:35]

,review_number,dominant_topic,original_text,topic_perc_contrib,tokens,num_tokens,keywords
30,8,Content & Authenticity,many are very similar and had to constantly ch...,0.3328,"[picture, person]",2,"three_kingdoms, series, fan, diplomacy, battle..."
31,9,Strategic Gameplay,this is the only total war game have not been ...,0.4194,"[finish, week, release, everytime, play, quality]",6,"faction, army, diplomacy, vassal, war, player,..."
32,9,Content & Authenticity,today decided to just uninstall it and move on...,0.3328,"[today, forcing]",2,"three_kingdoms, series, fan, diplomacy, battle..."
33,9,Strategic Gameplay,fan of the era itself or 2 have been too spoil...,0.3606,"[fan, era, spoilt, term, fan, rome, warhammer,...",10,"faction, army, diplomacy, vassal, war, player,..."
34,10,Strategic Gameplay,this game is amazing although the controls wer...,0.7893,"[control, makes_feel, china, chinese_culture, ...",5,"faction, army, diplomacy, vassal, war, player,..."


# Critical mistakes, easy to miss
I made the mistake of feeding my Phraser the wrong list of sentences earlier. I should have fed it the main training set instead of the test set. It did not form n-grams like "best_total_war" at all, which had negative impact when I first checked the data.

Also, more errors:

I have been training the ngram Phraser on the wrong dataset! I applied it correctly when training the LDA Model, but I didn't replicate it correctly when running the bigram Phraser on subsequent models. It did not combine 'shogun' and 'two' to form 'shogun_two' because I didn't take 'one', 'two', and 'three' out of the first run of stopwords each time.

I will take care of these breaking points because the program can still run, but the analysis will be broken due to an incorrectly trained Phraser.

# It's not perfect

SpaCy will read some nouns as verbs or other forms of the word, e.g. "map". It appears as a token in an earlier review, but in a later review the word "map" is missing from tokens.

Could be something to improve, but it could require hard-coding it so that it doesn't output this problem again.